# Solving Sudoku Puzzles
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

For this assignment you will need to submit:
1. The implementation for an agent which can solve sudoku puzzles – this notebook
 * You can use any algorithm you like, from the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned based on whether it can solve sudokus of varying difficulty
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section

### Choice of Algorithm
The choice of algorithm to solve sudoku puzzles is up to you. We expect you will use search techniques from the unit, but you could make something up yourself, or do some independent research to find something else. You will need to evaluate and balance the trade-off between how well suited you think the algorithm is and how difficult it is to write, but there is some advice below.

I suggest you implement *constraint satisfaction* as it is described in the unit material. You can use the code you have previously been given as a guide. A good implementation of a backtracking depth-first search with constraint propagation should be sufficient to get a good grade in the automated tests (roughly 60-70%).

You could also write a successful agent that uses the other search techniques you have seen in the unit so far: basic search, heuristic search, or local search. You may find these easier to implement, though they may perform less well. 

To get a high grade on this assignment will require a particularly efficient implementation of constraint satisfaction, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [1]:
import heapq
import math
import this

import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudo

## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested.

In [ ]:
def sudoku_solverdel(board):
    rows, cols, triples, empty_cells = defaultdict(set), defaultdict(set), defaultdict(set), []
    # populate init vals
    for r in range(9):
        for c in range(9):
            if board[r][c] != 0:
                val = board[r][c]
                if val in rows[r] or val in cols[c] or val in triples[(r//3,c//3)]:
                    return [[-1]*9]*9
                rows[r].add(board[r][c])
                cols[c].add(board[r][c])
                triples[(r // 3, c // 3)].add(board[r][c])
            else:
                empty_cells.append((r, c))

    def dfs(curr_board):
        if not empty_cells:
            return curr_board
        r, c = empty_cells.pop()
        grid = (r//3, c//3)

        for val in range(1, 10):
            if val not in rows[r] and val not in cols[c] and val not in triples[grid]:
                curr_board[r][c] = val
                rows[r].add(val)
                cols[c].add(val)
                triples[grid].add(val)

                # empty_cells.pop()

                if dfs(curr_board) is not None:
                    return curr_board
                else:
                    curr_board[r][c] = 0
                    rows[r].discard(val)
                    cols[c].discard(val)
                    triples[grid].discard(val)

                    empty_cells.append((r, c))
        return None

    state = dfs(board)
    return state if state is not None else [[-1]*9]*9

In [2]:
from typing import List
#
# class Cell:
#     def __init__(self, r: int, c: int, val: int):
#         self.row = r
#         self.col = c
#         self.val = val
#         self.grid = r // 3 + c // 3 # CALCULATE GRID LATER!
#
#     def is_empty(self):
#         return self.val == 0
#
#
# class PartialSudokuState:
#     def __init__(self, board: List[List[int]], n = 9):
#         # TODO: IF n is not a perfect square, throw error
#         self.n = 9
#         self.board = board
#         # self.row_to_vals = {row: board[row] for row in range(9)}
#         self.row_to_seen_vals = defaultdict(set)
#
#         self.col_to_seen_vals = defaultdict(set)
#         self.col_to_occupied_cells = defaultdict(set)
#         self.row_to_occupied_cells = defaultdict(set)
#         self.col_to_missing_rows = defaultdict(set)
#         self.col_to_missing_cells = defaultdict(set)
#
#         self.cols_with_min_missing_vals = {self.n} # tuple of list of
#
#         self.triple_grid = defaultdict(set)
#         self.min_col_count = float('inf')
#
#         self.populate_init_state()
#
#     # def get_col_with_min_missing(self):
#     #     pass
#
#     # WHAT IF SUDOKU IS ALREADY FULL OR TOTALLY EMPTY
#     def populate_init_state(self):
#         for col in range(self.n):
#             for row in range(self.n):
#                 val = self.board[row][col]
#                 # cell = Cell(row, col, val)
#                 if val == 0:
#                     self.col_to_missing_rows[col].add(row)
#                     # self.col_to_missing_cells[col].add(cell)
#                 else:
#                     if self.is_unsolvable(row, col, val):
#                         raise ValueError(f'Duplicate value found')
#                     # self.col_to_occupied_cells[col].add(cell)
#                     self.col_to_seen_vals[col].add(val)
#                     self.row_to_seen_vals[row].add(val)
#                     self.triple_grid[(row//3, col//3)].add(val)
#
#             # col with min missing val
#             col_count = len(self.col_to_missing_rows[col]) if col in self.col_to_missing_rows else float('inf')
#             if col_count < self.min_col_count:
#                 self.cols_with_min_missing_vals = {col}
#                 self.min_col_count = col_count
#             elif col_count == self.min_col_count:
#                 self.cols_with_min_missing_vals.add(col)
#
#     def is_unsolvable(self, row, col, val):
#         return val in self.col_to_seen_vals[col] or val in self.row_to_seen_vals[row] or val in self.triple_grid[(row//3, col//3)]
#
#
#     def choose_next_cell(self, col):
#         # choose row from valid set of rows in colToPossibleRows map
#         # for each col value, check which val is missing from only one row i.e present in only 1 row in rowToPossibleVals map. (func that takes in all poss rows and returns colVals and returns val missing in only 1 row. Perhaps return row too)
#         # get grid of row_col cell from cellTo3by3Grid map and also check if val is absent
#         # return (val, row_possibilities) if absent in grid, else move on to next val.
#         rows = self.col_to_missing_rows[col]
#         col_missing_vals = self.get_missing_values_from_col(col)
#         vals = []
#         min_count = self.n + 1
#
#         for val in col_missing_vals:
#             missing_count = 0
#             curr_missing_rows = []
#
#             for row in rows:
#                 row_seen_vals = self.row_to_seen_vals[row]
#                 if val not in row_seen_vals: # val not seen
#                     missing_count += 1
#                     curr_missing_rows.append(row)
#
#             if min_count == missing_count:
#                 vals.append((val, curr_missing_rows))
#             elif min_count > missing_count > 0:
#                 min_count = missing_count
#                 vals = [(val, curr_missing_rows)]
#
#         return vals
#
#         # TODO: IF NO COL, THEN STATE IS GOAL STATE
#         # return self.cols_with_min_missing_vals[0]
#
#     def is_goal_state(self):
#         return len(self.col_to_missing_rows) == 0
#
#     def get_missing_values_from_col(self, col):
#         missing_vals = set()
#         seen_vals = self.col_to_seen_vals[col]
#         for k in range(1, self.n + 1):
#             if k not in seen_vals:
#                 missing_vals.add(k)
#         return missing_vals
#
#     # def set_unique_val(self, row, col, val):
#     #     if self.is_unsolvable(row, col, val):
#     #         return None
#     #     self.board[row][col] = val
#
#
#     def set_val(self, row, col, val, new=True):
#         if self.is_unsolvable(row, col, val):
#             return None
#         new_board = self.board.copy() if new else self.board
#         new_board[row][col] = val
#
#         return PartialSudokuState(new_board)



# from collections import defaultdict
#
#
# def sudoku_solver0(sudoku):
#     """
#     Solves a Sudoku puzzle and returns its unique solution.
#
#     Input
#         sudoku : 9x9 numpy array
#             Empty cells are designated by 0.
#
#     Output
#         9x9 numpy array of integers
#             It contains the solution, if there is one. If there is no solution, all array entries should be -1.
#     """
#     # col_to_missing_vals = defaultdict(set)
#     # col_to_missing_rows = defaultdict(set)
#     # col_to_seen_vals = defaultdict(set)
#     # col_to_occupied_cells = defaultdict(set)
#     # row_to_occupied_cells = defaultdict(set)
#     # row_to_seen_vals = defaultdict(set)
#     #
#     # print(sudoku)
#     # row_to_vals = {row: sudoku[row] for row in range(9)}
#     #
#     #
#
#     # print('row_to_present_vals:', new_sudoku_state.row_to_seen_vals)
#     # print('col_to_seen_vals:', new_sudoku_state.col_to_seen_vals)
#     # # print('col_to_occupied_cells::', new_sudoku_state.col_to_occupied_cells)
#     # print('col_to_missing_rows:', new_sudoku_state.col_to_missing_rows)
#     # print('cols_with_min_missing_vals:',new_sudoku_state.cols_with_min_missing_vals, 'count:', new_sudoku_state.min_col_count)
#     ### YOUR CODE HERE
#
#     # new_sudoku = sudoku.copy()
#     # print('new sudoku:', sudoku)
#     # generate colToPossibleVals map. Derive colWithMinPossibleVals
#     # generate rowToPossibleVals map.
#     # generate colToPossibleRows map. DONE
#     # generate cellTo3by3Grid map
#     def depth_first_search(sudoku_state):
#         # if goal state, return new_sudoku
#         # goal state is obtained when colToPossibleVals map or/and rowToPossibleVals map are empty
#         if sudoku_state.is_goal_state():
#             return sudoku_state
#         # choose col: this should be the col with the min no of possible vals [colWithMinPossibleVals]
#         cols = sudoku_state.cols_with_min_missing_vals
#         # choose row from valid set of rows in colToPossibleRows map
#         # for each col value, check which val is missing from only one row i.e present in only 1 row in rowToPossibleVals map. (func that takes in all poss rows and returns colVals and returns val missing in only 1 row. Perhaps return row too)
#         # get grid of row_col cell from cellTo3by3Grid map and also check if val is absent
#         # return (val, row_possibilities) if absent in grid, else move on to next val.
#         for col in cols:
#             for val, rows in sudoku_state.choose_next_cell(col):
#                 # print('choosing next cell. col:', col,'val:', val, 'row:', rows )
#                 if len(rows) == 1:
#                     new_state = sudoku_state.set_val(rows[0], col, val, False)
#                     if new_state:
#                         deep_state = depth_first_search(new_state)
#                     break
#                 for row in rows:
#                     new_board = False if len(rows) == 1 else True
#
#                     new_state = sudoku_state.set_val(row, col, val, new_board)
#
#                     if new_state:
#                         # print('new state:', new_state.board)
#                         deep_state = depth_first_search(new_state)
#                         if deep_state is not None and deep_state.is_goal_state():
#                             return deep_state
#         return None
#
#     # print(sudoku)
#     try:
#         initial_state = PartialSudokuState(sudoku)
#         goal_state = depth_first_search(initial_state)
#         if not goal_state:
#             raise ValueError('Goal state solution was never reached')
#         return goal_state.board
#     except ValueError as ve:
#         print(ve)
#         return [[-1] * 9 ] * 9
#
#
#
#     # if val's possibilty is ? 1 row (one col val can be placed in >1 row)
#         # perform dfs with val on all possibilities [maybe look to change approach later - to continually check for anothe val in col or choose another col)
#         # derive new partial state for each dfs and perform below
#     # else
#         # remove val from col & rows in maps [this is similar to placing the val on the row_col cell]
#         # add no to cellTo3by3Grid map
#         # update min col
#     # BACK TO CHOOSE COL
#
#     # return new_sudoku
#
# # import numpy as np
# #
# # # Load sudokus
# # sudokus = np.load("data/very_easy_puzzle.npy")
# # sudoku = sudokus[0].copy()
# # sudoku_solver(sudoku)

In [68]:
# def sudoku_solver(board):
#     rows, cols, triples, empty_cells = defaultdict(set), defaultdict(set), defaultdict(set), []
#     # populate init vals
#     for r in range(9):
#         for c in range(9):
#             if board[r][c] != 0:
#                 rows[r].add(board[r][c])
#                 cols[c].add(board[r][c])
#                 triples[(r // 3, c // 3)].add(board[r][c])
#             else:
#                 empty_cells.append((r, c))
#
#     def dfs(curr_board):
#         print(curr_board)
#         if not empty_cells:
#             return curr_board
#         r, c = empty_cells[-1]
#         grid = (r//3, c//3)
#         for val in range(1, 10):
#             if val not in rows[r] and val not in cols[c] and val not in triples[grid]:
#                 curr_board[r][c] = val
#                 rows[r].add(val)
#                 cols[c].add(val)
#                 triples[grid].add(val)
#
#                 empty_cells.pop()
#
#                 if dfs(curr_board) is not None:
#                     return curr_board
#                 else:
#                     curr_board[r][c] = 0
#                     rows[r].discard(val)
#                     cols[c].discard(val)
#                     triples[grid].discard(val)
#
#                     empty_cells.append((r, c))
#         return None
#
#     dfs(board)

In [74]:
from heapq import heapify
from collections import defaultdict
from typing import List
import copy

class SudokuState:

    def __init__(self, board: List[List[int]], n = 9):

        """
        Initialize sudoku state
        Throws Value error if duplicate values are found in same rows, cols or sub grid.

        Parameters
        ----------
        board : List[List[int]]
            The sudoku board.
        n : int, optional
            The size of the sudoku board (default is None). This is used so the sudoku can be extended for other suitable board sizes.
        """
        self.n = n
        self.n_sq_root = math.sqrt(self.n)
        self.board = board

        self.row_to_seen_vals = defaultdict(set)
        self.col_to_seen_vals = defaultdict(set)
        self.sub_grid = defaultdict(set)

        self.empty_cells = []

        self._initialize_state()

    def _initialize_state(self):
        """
        Helper method to initialize state instance variables.

        Raises
        ------
        ValueError
            If duplicate value is found either in row, column or sub grid.
        """
        for row in range(self.n):
            for col in range(self.n):
                val = self.board[row][col]
                if val == 0:
                    self.empty_cells.append((row, col))
                else:
                    if self.is_unsolvable(row, col, val):
                        raise ValueError(f'Duplicate value found')
                    self.col_to_seen_vals[col].add(val)
                    self.row_to_seen_vals[row].add(val)
                    self.sub_grid[self._grid_key(row, col)].add(val)

    def is_unsolvable(self, row, col, val):
        """
        Returns True if this sudoku state can have a valid solution or False otherwise
        :param row: row to check for
        :param col: column to check for
        :param val: value which we want to determine if it can be placed in the (row, col) cell
        :return: True if the value is seen in column, row or sub grid. False otherwise
        """
        return val in self.col_to_seen_vals[col] or val in self.row_to_seen_vals[row] or val in self.sub_grid[self._grid_key(row, col)]

    def is_goal_state(self):
        """This state is a goal state if the length of the empty cells is zero
        :return: True if there are no more cells that are empty or False otherwise
        """
        return len(self.empty_cells) == 0

    def place_digit(self, row, col, val):
        """
        Place digit value in the specified the (row, col) coordinate of the state's board
        :param row: row where digit should be placed
        :param col: col where digit should be placed
        :param val: digit to be placed
        """
        self.board[row][col] = val
        # remove the last cell
        self.empty_cells.pop()

        # update the seen values in the rows and cols
        self.row_to_seen_vals[row].add(val)
        self.col_to_seen_vals[col].add(val)
        self.sub_grid[self._grid_key(row, col)].add(val)

    def revert_state(self, row, col, val: int):
        """
        Revert the state of the sudoku during backtracking
        :param row: row to be reverted
        :param col: column to be reverted
        :param val: digit in the (row, col) cell to be reverted
        """
        self.empty_cells.append((row, col))

        # remove the values previously added to the data structures when the digit was placed
        self.row_to_seen_vals[row].discard(val)
        self.col_to_seen_vals[col].discard(val)
        self.sub_grid[self._grid_key(row, col)].discard(val)

    def _grid_key(self, row, col):
        """
        Get key for the subgrid that contains the (row, col) cell
        :param row: The row contained in the grid
        :param col: The column contained in the grid
        :return: The key of the grid that contains the (row, col) cell
        """
        return row // self.n_sq_root, col // self.n_sq_root

    def choose_cell(self):
        """
        Choose the next (row, col) cell to place digit in.
        The last empty cell is chosen so that digit can be placed and discarded as necessary with O(1) time complexity
        :return: (row, col) tuple
        """
        return self.empty_cells[-1]


In [77]:
import math

def sudoku_solver(sudoku):

    def depth_first_search(sudoku_state):
        if sudoku_state.is_goal_state():
            return sudoku_state

        row, col = sudoku_state.choose_cell()

        for digit in range(1, 10):
            # check if a digit can be put in the cell. If not valid, backtrack
            if not sudoku_state.is_unsolvable(row, col, digit):
                sudoku_state.place_digit(row, col, digit)
                deep_state = depth_first_search(sudoku_state)
                if not deep_state:
                    # backtrack and move forward to next val
                    sudoku_state.revert_state(row, col, digit)
                else:
                    return deep_state
        return None

    invalid_sudoku = [[-1] * 9] * 9

    # if not valid_sudoku_size(sudoku):
    #     return invalid_sudoku

    try:
        initial_state = SudokuState(sudoku)
        goal_state = depth_first_search(initial_state)
        if not goal_state:
            raise ValueError('Goal state solution was never reached')
        return goal_state.board
    except ValueError as e:
        print(e)
        return invalid_sudoku

def valid_sudoku_size(sudoku):
    sqrt = math.isqrt(sudoku.size)
    if sqrt * sqrt != sudoku.size:
        print("Invalid sudoku size. Sudoku must be a perfect square with column and row of same size")
        return False
    return True


All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 30 seconds each on the test machine to count as successful, but you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [78]:
import numpy as np
# SET BACK TO TRUE AFTERWARDS
SKIP_TESTS = False

if not SKIP_TESTS:
    import time
    # difficulties = ['very_easy', 'easy', 'medium', 'hard']
    difficulties = ['hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break

Testing hard sudokus
This is hard sudoku number 0
[[0 0 0 0 0 7 5 4 0]
 [9 0 6 0 5 0 0 3 0]
 [0 0 0 0 0 0 2 0 0]
 [2 0 0 0 0 0 7 9 0]
 [0 0 3 0 4 1 0 0 0]
 [7 0 0 0 0 0 0 5 0]
 [0 3 0 0 0 4 0 2 0]
 [0 9 4 1 0 0 0 0 0]
 [0 0 0 5 9 0 0 0 4]]
Goal state solution was never reached
This is your solution for hard sudoku number 0
[[-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 1.518261999999993 seconds to solve.

This is hard sudoku number 1
[[1 0 0 7 0 0 0 0 0]
 [0 3 2 0 0 0 0 0 0]
 [0 0 0 6 0 0 0 0 0]
 [0 8 0 0 0 2 0 7 0]
 [5 0 7 0 0 1 0 0 0]
 [0 0 0 0 0 3 6 1 0]
 [7 0 0 0 0 0 2 0 9]
 [0 0 0 0 5 0 0 0 0]
 [3 0 0 0 0 4 0 0 5]]
Goal state

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [7]:
import sys
import pathlib

fail = False;

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the cell above.")
    
p1 = pathlib.Path('./readme.txt')
p2 = pathlib.Path('./readme.md')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("You must include a separate file called readme.txt or readme.md in your submission.")
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named sudoku.ipynb")
    
if "sudoku_solver" not in dir():
    fail = True;
    print("You must include a function called sudoku_solver which accepts a numpy array.")
else: 
    sudoku = np.load("data/very_easy_puzzle.npy")[0]
    solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(sudoku), solution):
        print("Warning:")
        print("Your sudoku_solver function does not correctly solve the first sudoku.")
        print()
        print("Your assignment is unlikely to get any marks from the autograder. While we will")
        print("try to check it manually to assign some partial credit, we encourage you to ask")
        print("for help on the forum or directly to a tutor.")
        print()
        print("Please use the readme file to explain your code anyway.")
    
if fail:
    print()
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

You must set the SKIP_TESTS constant to True in the cell above.
You must include a separate file called readme.txt or readme.md in your submission.
Invalid sudoku size. Sudoku must be a perfect square with column and row of same size
Your sudoku_solver function does not correctly solve the first sudoku.

Your assignment is unlikely to get any marks from the autograder. While we will
try to check it manually to assign some partial credit, we encourage you to ask
for help on the forum or directly to a tutor.

Please use the readme file to explain your code anyway.



Your submission is not ready! Please read and follow the instructions above.

In [8]:
# This is a TEST CELL. Do not delete or change.